In [1]:
import torch
import torch.nn as nn

import torchvision.transforms as transforms
import matplotlib.pyplot as plt

import os
import numpy as np
from IPython import display
import pandas as pd
import torch
import torchaudio
import torchaudio.functional as F
import torchaudio.transforms as T
import torch.nn.functional as F
import torchvision.models as models # resnet18 pretrained model
from efficientnet_pytorch import EfficientNet

/usr/local/lib/python3.9/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:


###pytorch functions imported from https://pytorch.org/tutorials/beginner/audio_preprocessing_tutorial.html
########pytorch audio formatting functions

import os
import math


import librosa


import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import time
from IPython.display import Audio, display

[width, height] = matplotlib.rcParams['figure.figsize']
if width < 10:
  matplotlib.rcParams['figure.figsize'] = [width * 2.5, height]

def _get_sample(path, resample=None):
  effects = [
    ["remix", "1"]
  ]
  if resample:
    effects.extend([
      ["lowpass", f"{resample // 2}"],
      ["rate", f'{resample}'],
    ])
  return torchaudio.sox_effects.apply_effects_file(path, effects=effects)

def get_speech_sample(*, resample=None):
  return _get_sample(SAMPLE_WAV_SPEECH_PATH, resample=resample)

def get_sample(*, resample=None):
  return _get_sample(SAMPLE_WAV_PATH, resample=resample)

def get_rir_sample(*, resample=None, processed=False):
  rir_raw, sample_rate = _get_sample(SAMPLE_RIR_PATH, resample=resample)
  if not processed:
    return rir_raw, sample_rate
  rir = rir_raw[:, int(sample_rate*1.01):int(sample_rate*1.3)]
  rir = rir / torch.norm(rir, p=2)
  rir = torch.flip(rir, [1])
  return rir, sample_rate

def get_noise_sample(*, resample=None):
  return _get_sample(SAMPLE_NOISE_PATH, resample=resample)

def print_stats(waveform, sample_rate=None, src=None):
  if src:
    print("-" * 10)
    print("Source:", src)
    print("-" * 10)
  if sample_rate:
    print("Sample Rate:", sample_rate)
  print("Shape:", tuple(waveform.shape))
  print("Dtype:", waveform.dtype)
  print(f" - Max:     {waveform.max().item():6.3f}")
  print(f" - Min:     {waveform.min().item():6.3f}")
  print(f" - Mean:    {waveform.mean().item():6.3f}")
  print(f" - Std Dev: {waveform.std().item():6.3f}")
  print()
  print(waveform)
  print()

def plot_waveform(waveform, sample_rate, title="Waveform", xlim=None, ylim=None):
  waveform = waveform.numpy()

  num_channels, num_frames = waveform.shape
  time_axis = torch.arange(0, num_frames) / sample_rate

  figure, axes = plt.subplots(num_channels, 1)
  if num_channels == 1:
    axes = [axes]
  for c in range(num_channels):
    axes[c].plot(time_axis, waveform[c], linewidth=1)
    axes[c].grid(True)
    if num_channels > 1:
      axes[c].set_ylabel(f'Channel {c+1}')
    if xlim:
      axes[c].set_xlim(xlim)
    if ylim:
      axes[c].set_ylim(ylim)
  figure.suptitle(title)
  plt.show(block=False)

def plot_specgram(waveform, sample_rate, title="Spectrogram", xlim=None):
  waveform = waveform.numpy()

  num_channels, num_frames = waveform.shape
  time_axis = torch.arange(0, num_frames) / sample_rate

  figure, axes = plt.subplots(num_channels, 1)
  if num_channels == 1:
    axes = [axes]
  for c in range(num_channels):
    axes[c].specgram(waveform[c], Fs=sample_rate)
    if num_channels > 1:
      axes[c].set_ylabel(f'Channel {c+1}')
    if xlim:
      axes[c].set_xlim(xlim)
  figure.suptitle(title)
  plt.show(block=False)

def play_audio(waveform, sample_rate):
  waveform = waveform.numpy()

  num_channels, num_frames = waveform.shape
  if num_channels == 1:
    display(Audio(waveform[0], rate=sample_rate))
  elif num_channels == 2:
    display(Audio((waveform[0], waveform[1]), rate=sample_rate))
  else:
    raise ValueError("Waveform with more than 2 channels are not supported.")

def inspect_file(path):
  print("-" * 10)
  print("Source:", path)
  print("-" * 10)
  print(f" - File size: {os.path.getsize(path)} bytes")
  print(f" - {torchaudio.info(path)}")

def plot_spectrogram(spec, title=None, ylabel='freq_bin', aspect='auto', xmax=None):
  fig, axs = plt.subplots(1, 1)
  axs.set_title(title or 'Spectrogram (db)')
  axs.set_ylabel(ylabel)
  axs.set_xlabel('frame')
  im = axs.imshow(librosa.power_to_db(spec), origin='lower', aspect=aspect)
  if xmax:
    axs.set_xlim((0, xmax))
  fig.colorbar(im, ax=axs)
  plt.show(block=False)

def plot_mel_fbank(fbank, title=None):
  fig, axs = plt.subplots(1, 1)
  axs.set_title(title or 'Filter bank')
  axs.imshow(fbank, aspect='auto')
  axs.set_ylabel('frequency bin')
  axs.set_xlabel('mel bin')
  plt.show(block=False)

def get_spectrogram(
    n_fft = 400,
    win_len = None,
    hop_len = None,
    power = 2.0,
):
  waveform, _ = get_speech_sample()
  spectrogram = T.Spectrogram(
      n_fft=n_fft,
      win_length=win_len,
      hop_length=hop_len,
      center=True,
      pad_mode="reflect",
      power=power,
  )
  return spectrogram(waveform)

def plot_pitch(waveform, sample_rate, pitch):
  figure, axis = plt.subplots(1, 1)
  axis.set_title("Pitch Feature")
  axis.grid(True)

  end_time = waveform.shape[1] / sample_rate
  time_axis = torch.linspace(0, end_time,  waveform.shape[1])
  axis.plot(time_axis, waveform[0], linewidth=1, color='gray', alpha=0.3)

  axis2 = axis.twinx()
  time_axis = torch.linspace(0, end_time, pitch.shape[1])
  ln2 = axis2.plot(
      time_axis, pitch[0], linewidth=2, label='Pitch', color='green')

  axis2.legend(loc=0)
  plt.show(block=False)

def plot_kaldi_pitch(waveform, sample_rate, pitch, nfcc):
  figure, axis = plt.subplots(1, 1)
  axis.set_title("Kaldi Pitch Feature")
  axis.grid(True)

  end_time = waveform.shape[1] / sample_rate
  time_axis = torch.linspace(0, end_time,  waveform.shape[1])
  axis.plot(time_axis, waveform[0], linewidth=1, color='gray', alpha=0.3)

  time_axis = torch.linspace(0, end_time, pitch.shape[1])
  ln1 = axis.plot(time_axis, pitch[0], linewidth=2, label='Pitch', color='green')
  axis.set_ylim((-1.3, 1.3))

  axis2 = axis.twinx()
  time_axis = torch.linspace(0, end_time, nfcc.shape[1])
  ln2 = axis2.plot(
      time_axis, nfcc[0], linewidth=2, label='NFCC', color='blue', linestyle='--')

  lns = ln1 + ln2
  labels = [l.get_label() for l in lns]
  axis.legend(lns, labels, loc=0)
  plt.show(block=False)

DEFAULT_OFFSET = 201
SWEEP_MAX_SAMPLE_RATE = 48000
DEFAULT_LOWPASS_FILTER_WIDTH = 6
DEFAULT_ROLLOFF = 0.99
DEFAULT_RESAMPLING_METHOD = 'sinc_interpolation'

def _get_log_freq(sample_rate, max_sweep_rate, offset):
  """Get freqs evenly spaced out in log-scale, between [0, max_sweep_rate // 2]

  offset is used to avoid negative infinity `log(offset + x)`.

  """
  half = sample_rate // 2
  start, stop = math.log(offset), math.log(offset + max_sweep_rate // 2)
  return torch.exp(torch.linspace(start, stop, sample_rate, dtype=torch.double)) - offset

def _get_inverse_log_freq(freq, sample_rate, offset):
  """Find the time where the given frequency is given by _get_log_freq"""
  half = sample_rate // 2
  return sample_rate * (math.log(1 + freq / offset) / math.log(1 + half / offset))

def _get_freq_ticks(sample_rate, offset, f_max):
  # Given the original sample rate used for generating the sweep,
  # find the x-axis value where the log-scale major frequency values fall in
  time, freq = [], []
  for exp in range(2, 5):
    for v in range(1, 10):
      f = v * 10 ** exp
      if f < sample_rate // 2:
        t = _get_inverse_log_freq(f, sample_rate, offset) / sample_rate
        time.append(t)
        freq.append(f)
  t_max = _get_inverse_log_freq(f_max, sample_rate, offset) / sample_rate
  time.append(t_max)
  freq.append(f_max)
  return time, freq

def plot_sweep(waveform, sample_rate, title, max_sweep_rate=SWEEP_MAX_SAMPLE_RATE, offset=DEFAULT_OFFSET):
  x_ticks = [100, 500, 1000, 5000, 10000, 20000, max_sweep_rate // 2]
  y_ticks = [1000, 5000, 10000, 20000, sample_rate//2]

  time, freq = _get_freq_ticks(max_sweep_rate, offset, sample_rate // 2)
  freq_x = [f if f in x_ticks and f <= max_sweep_rate // 2 else None for f in freq]
  freq_y = [f for f in freq if f >= 1000 and f in y_ticks and f <= sample_rate // 2]

  figure, axis = plt.subplots(1, 1)
  axis.specgram(waveform[0].numpy(), Fs=sample_rate)
  plt.xticks(time, freq_x)
  plt.yticks(freq_y, freq_y)
  axis.set_xlabel('Original Signal Frequency (Hz, log scale)')
  axis.set_ylabel('Waveform Frequency (Hz)')
  axis.xaxis.grid(True, alpha=0.67)
  axis.yaxis.grid(True, alpha=0.67)
  figure.suptitle(f'{title} (sample rate: {sample_rate} Hz)')
  plt.show(block=True)

def get_sine_sweep(sample_rate, offset=DEFAULT_OFFSET):
    max_sweep_rate = sample_rate
    freq = _get_log_freq(sample_rate, max_sweep_rate, offset)
    delta = 2 * math.pi * freq / sample_rate
    cummulative = torch.cumsum(delta, dim=0)
    signal = torch.sin(cummulative).unsqueeze(dim=0)
    return signal

def benchmark_resample(
    method,
    waveform,
    sample_rate,
    resample_rate,
    lowpass_filter_width=DEFAULT_LOWPASS_FILTER_WIDTH,
    rolloff=DEFAULT_ROLLOFF,
    resampling_method=DEFAULT_RESAMPLING_METHOD,
    beta=None,
    librosa_type=None,
    iters=5
):
  if method == "functional":
    begin = time.time()
    for _ in range(iters):
      F.resample(waveform, sample_rate, resample_rate, lowpass_filter_width=lowpass_filter_width,
                 rolloff=rolloff, resampling_method=resampling_method)
    elapsed = time.time() - begin
    return elapsed / iters
  elif method == "transforms":
    resampler = T.Resample(sample_rate, resample_rate, lowpass_filter_width=lowpass_filter_width,
                           rolloff=rolloff, resampling_method=resampling_method, dtype=waveform.dtype)
    begin = time.time()
    for _ in range(iters):
      resampler(waveform)
    elapsed = time.time() - begin
    return elapsed / iters
  elif method == "librosa":
    waveform_np = waveform.squeeze().numpy()
    begin = time.time()
    for _ in range(iters):
      librosa.resample(waveform_np, sample_rate, resample_rate, res_type=librosa_type)
    elapsed = time.time() - begin
    return elapsed / iters

In [3]:
#dataset processing and splitting
dfInput=pd.read_csv('/code/dataset/FakeAVCeleb/MergedMetadataLabeled.csv')
dfCut=dfInput[['wavLocation','Audio_Label']]
dfCut2=dfInput[['wavLocation','Audio_Label','path']]
wavDir=dfCut2['wavLocation']
labelsdf=dfCut2[['wavLocation','Audio_Label']]
dfFake=dfCut[dfCut['Audio_Label']==1].reset_index(drop=True)
dfReal=dfCut[dfCut['Audio_Label']==0].reset_index(drop=True)
dfTraining=pd.concat([dfReal[0:8000],dfFake[0:8000]]).reset_index(drop=True)
dfValidation=pd.concat([dfReal[8000:],dfFake[8000:]]).reset_index(drop=True)
labelsTrain=dfTraining['Audio_Label']
labelsVal=dfValidation['Audio_Label']

In [4]:
#training data prep
#create tensor of wav form, and its sample rate



dfTensor=[]
#melspectrogram

n_fft = 1024
win_length = None
hop_length = 512
n_mels = 64 #downsample from 128 to something lower
for i in range(0,len(dfTraining)):
    audioPath=dfTraining['wavLocation'][i]
    waveform, sample_rate = torchaudio.load(audioPath)


    mel_spectrogram = T.MelSpectrogram(
        sample_rate=sample_rate,
        n_fft=n_fft,
        win_length=win_length,
        hop_length=hop_length,
        center=True,
        pad_mode="reflect",
        power=2.0,
        norm='slaney',
        onesided=True,
        n_mels=n_mels,
        mel_scale="htk",
    )

    melspec = mel_spectrogram(waveform)
    dfTensor+=[melspec]
#create tensor of wav form, and its sample rate
train_data=[]
for i in range(0,len(dfTraining)):
    train_data.append([dfTensor[i],labelsTrain[i]])

#padding to create equal length Tensors
target_len = 275 #make samples smaller
train_data_pad=[]
for i in range(0,len(train_data)):
    x=train_data[i][0]
    x = F.pad(x, (target_len - x.size(2), 0))
    train_data_pad.append([x,labelsTrain[i]])


trainloader = torch.utils.data.DataLoader(train_data_pad, shuffle=True, batch_size=100)
i1, l1 = next(iter(trainloader))
print(i1.shape)

torch.Size([100, 2, 64, 275])


In [5]:
#validation data prep
#create tensor of wav form, and its sample rate



dfTensor=[]
#melspectrogram

n_fft = 1024
win_length = None
hop_length = 512
n_mels = 64#change from 128
for i in range(0,len(dfValidation)):
    audioPath=dfValidation['wavLocation'][i]
    waveform, sample_rate = torchaudio.load(audioPath)


    mel_spectrogram = T.MelSpectrogram(
        sample_rate=sample_rate,
        n_fft=n_fft,
        win_length=win_length,
        hop_length=hop_length,
        center=True,
        pad_mode="reflect",
        power=2.0,
        norm='slaney',
        onesided=True,
        n_mels=n_mels,
        mel_scale="htk",
    )

    melspec = mel_spectrogram(waveform)
    dfTensor+=[melspec]
#create tensor of wav form, and its sample rate
validation_data=[]
for i in range(0,len(dfValidation)):
    validation_data.append([dfTensor[i],labelsVal[i]])

#padding to create equal length Tensors
target_len = 275
validation_data_pad=[]
for i in range(0,len(validation_data)):
    x=validation_data[i][0]
    x = F.pad(x, (target_len - x.size(2), 0))
    validation_data_pad.append([x,labelsVal[i]])


validationloader = torch.utils.data.DataLoader(validation_data_pad, shuffle=True, batch_size=100)
i2, l2 = next(iter(validationloader))
print(i2.shape)

torch.Size([100, 2, 64, 275])


In [6]:


class MyModel(nn.Module):

    def __init__(self):
        super(MyModel, self).__init__()
        # Convert 2 channel to 3 channel to be able to send to resnet18
        self.conv1 = nn.Conv2d(2, 3, kernel_size=3, padding=1)
        self.base_model = EfficientNet.from_pretrained('efficientnet-b2')
        self.fc2 = nn.Linear(1000, 2) # 264 different birds

    def forward(self, x):
        
        x = self.conv1(x)
        x = self.base_model(x)
        x = self.fc2(x)
        
        return x
        



In [7]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
model = MyModel().to(device)

Loaded pretrained weights for efficientnet-b2


In [8]:
def get_accuracy(y_pred, y_actual):
    y_pred_ = y_pred.argmax(1).detach().cpu().numpy()
    y_actual_ = y_actual.numpy()
    
    return np.mean(y_pred_ == y_actual_) * 100.0
criterion = nn.CrossEntropyLoss()


opt = torch.optim.Adam(model.parameters(), lr=0.0001)



EPOCHS = 5




In [9]:
running_acc = 0
for epoch in range(0, EPOCHS):
    print('\nTraining: \n')
    model.train()
    for b, data in enumerate(trainloader):
        #x=data[0]
        #y=data[1]
        opt.zero_grad()

        y_pred = model(data[0].to(device))
        loss = criterion(y_pred, data[1].to(device))
        loss.backward()

        opt.step()
        
        acc = get_accuracy(y_pred, data[1].cpu())
        
        running_acc = running_acc * 0.9 + acc * 0.1

        print('\rEpoch: {}/{}, \
        batch: {}/{}, \
        loss: {:4f}, \
        running_acc: {:.4f}'.format(epoch+1, EPOCHS, b+1, len(trainloader), loss.item(), running_acc),  end=' ')
    
    print('\nValidation: \n')
    running_acc = 0
    mean_acc = 0
    model.eval()
    for b, y in enumerate(validationloader):
        #x=data[0]
        #y=data[1]
            #(x,y)
        y_pred = y_pred = model(data[0].to(device))

        loss = criterion(y_pred, data[1].to(device))
        acc = get_accuracy(y_pred, data[1])

        running_acc = running_acc * 0.9 + acc * 0.1
        mean_acc = mean_acc + acc

        print('\rbatch: {}/{}, \
        loss: {:4f}, \
        acc: {:.4f}'.format(b+1, len(validationloader), loss.item(), running_acc),  end=' ')
    mean_acc /= len(validationloader)
    print('Mean accuracy: ', mean_acc)


Training: 

Epoch: 1/5,         batch: 160/160,         loss: 0.066284,         running_acc: 97.3612 
Validation: 

batch: 56/56,         loss: 0.472759,         acc: 78.7836 Mean accuracy:  79.0

Training: 

Epoch: 2/5,         batch: 160/160,         loss: 0.014310,         running_acc: 98.9265 
Validation: 

batch: 56/56,         loss: 0.023043,         acc: 99.7261 Mean accuracy:  100.0

Training: 

Epoch: 3/5,         batch: 160/160,         loss: 0.003487,         running_acc: 98.9931 
Validation: 

batch: 56/56,         loss: 0.005041,         acc: 99.7261 Mean accuracy:  100.0

Training: 

Epoch: 4/5,         batch: 160/160,         loss: 0.002839,         running_acc: 99.4110 
Validation: 

batch: 56/56,         loss: 0.000881,         acc: 99.7261 Mean accuracy:  100.0

Training: 

Epoch: 5/5,         batch: 160/160,         loss: 0.002962,         running_acc: 99.4850 
Validation: 

batch: 56/56,         loss: 0.006583,         acc: 99.7261 Mean accuracy:  100.0
